In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
import pickle
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
ResNet50_MODEL=tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                               include_top=False,
                                               weights="imagenet")

In [4]:
len(ResNet50_MODEL.layers)

175

In [5]:
ResNet50_MODEL.layers[15].name

'conv2_block1_0_bn'

In [6]:
for layer in ResNet50_MODEL.layers:
    layer.trainable=True

In [7]:
ResNet50_MODEL.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [8]:
batch_size = 64
l1_factor = 0.0001
l2_factor = 0.001
dropout = 0.5
learning_rate = 0.0001
target_size_image_shape = (224, 224)

In [9]:
model=tf.keras.models.Sequential([
                                  ResNet50_MODEL,
                                  tf.keras.layers.GlobalAveragePooling2D(),
                                  tf.keras.layers.Dropout(dropout, name='Dropout_Regularization1'),
                                  tf.keras.layers.Dense(1724, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
Dropout_Regularization1 (Dro (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1724)              3532476   
Total params: 27,120,188
Trainable params: 27,067,068
Non-trainable params: 53,120
_________________________________________________________________


In [10]:
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
train_data_dir = '../datasets/group10_set_224/set_224/train/'
valid_data_dir = '../datasets/group10_set_224/set_224/valid/'

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1.0/255)

In [12]:
#flow training images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size_image_shape,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='categorical'
)

Found 122251 images belonging to 1724 classes.


In [13]:
valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=target_size_image_shape,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='categorical'
)

Found 31031 images belonging to 1724 classes.


In [14]:
history = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = train_generator.n // batch_size,
    validation_data = valid_generator,
    validation_steps = valid_generator.n // batch_size,
    epochs = 100,
    workers = 16,
    verbose=1
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
1910/1910 [==============================] - 595s 312ms/step - loss: 6.1552 - acc: 0.0638 - val_loss: 4.1203 - val_acc: 0.2445
Epoch 2/100
1910/1910 [==============================] - 596s 312ms/step - loss: 3.9397 - acc: 0.2658 - val_loss: 3.0666 - val_acc: 0.4060
Epoch 3/100
1910/1910 [==============================] - 597s 312ms/step - loss: 3.1232 - acc: 0.3842 - val_loss: 2.7941 - val_acc: 0.4553
Epoch 4/100
1910/1910 [==============================] - 595s 311ms/step - loss: 2.6427 - acc: 0.4593 - val_loss: 2.5795 - val_acc: 0.4929
Epoch 5/100
1910/1910 [==============================] - 594s 311ms/step - loss: 2.3010 - acc: 0.5148 - val_loss: 2.2868 - val_acc: 0.5433
Epoch 6/100
1910/1910 [==============================] - 597s 313ms/step - loss: 2.0368 - acc: 0.5615 - val_loss: 2.3373 - val_acc: 0.5382
Epoch 7/100
1910/1910 [==============================] - 594s 311ms/step - loss: 1.8253 - 

Epoch 58/100
1910/1910 [==============================] - 590s 309ms/step - loss: 0.1517 - acc: 0.9559 - val_loss: 2.7576 - val_acc: 0.6534
Epoch 59/100
1910/1910 [==============================] - 589s 309ms/step - loss: 0.1491 - acc: 0.9576 - val_loss: 2.9173 - val_acc: 0.6454
Epoch 60/100
1910/1910 [==============================] - 590s 309ms/step - loss: 0.1504 - acc: 0.9562 - val_loss: 2.9401 - val_acc: 0.6585
Epoch 61/100
1910/1910 [==============================] - 590s 309ms/step - loss: 0.1423 - acc: 0.9588 - val_loss: 2.8717 - val_acc: 0.6557
Epoch 62/100
1910/1910 [==============================] - 589s 309ms/step - loss: 0.1419 - acc: 0.9588 - val_loss: 2.7559 - val_acc: 0.6524
Epoch 63/100
1910/1910 [==============================] - 589s 309ms/step - loss: 0.1402 - acc: 0.9595 - val_loss: 2.6524 - val_acc: 0.6748
Epoch 64/100
1910/1910 [==============================] - 597s 313ms/step - loss: 0.1339 - acc: 0.9612 - val_loss: 3.0027 - val_acc: 0.6458
Epoch 65/100
1910/19

KeyboardInterrupt: 

In [ ]:
sfsdfsdsdfs

In [ ]:
history.model.save('group5_set224_resnet50_NO_imagenet_weights_07132020')

In [ ]:
tf.keras.models.save_model(history.model, 'group5_set224_resnet50_07132020.h5', include_optimizer=True)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training accuracy')
plt.figure()

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training Loss')
plt.figure()

In [ ]:
gsedrfgdffg

### Predict Image

In [ ]:
indices_to_class_labels_dict = {value : key for key, value in train_generator.class_indices.items()}

In [ ]:
print(len(indices_to_class_labels_dict))
print(indices_to_class_labels_dict)

In [ ]:
with open("indices_to_class_labels_dict.json", "wb") as pickle_file:
    pickle.dump(indices_to_class_labels_dict, pickle_file)

In [2]:
def resize_image(path, new_width, new_height):
    image = Image.open(path)
    image = ImageOps.fit(image, (new_width, new_height), Image.ANTIALIAS)
    return image

In [3]:
image = resize_image("test_images\\e0dc68db51a90c10.jpg", 224, 224)
image

NameError: name 'Image' is not defined

In [ ]:
np_image = np.array(image)
print(np_image.shape)
np_image = np_image/255
image = np.expand_dims(np_image, axis=0)
result = history.model.predict(image)
predicted_class = indices_to_class_labels_dict[np.argmax(result)]
print("Identified landmark for image is : {}".format(predicted_class))

In [ ]:
history.model.predict(image)

In [ ]:
print(result.sum())